# 1. Import and Install Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import ipywidgets as widgets

# 2. Keypoints using MP Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Split Dataset 80:20

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU
from tensorflow.keras.callbacks import TensorBoard

actions = np.array(['ปกติ','หิว', 'ยิ้ม','โกรธ','ร้องไห้','เสียใจ', 
                    'สนุก', 'ขอโทษ', 'ขอบคุณ','แฟน', 'เพื่อน', 
                    'คน', 'คนชรา','เด็ก', 'ทารก', 'ผู้ชาย', 
                    'ผู้หญิง','ผู้ใหญ่', 'วัยรุ่น', 'หนุ่มสาว', 'น้อง'
                   ])
DATA_PATH = 'E:/Downloads/ActionDetectionforSignLanguage-main/dataset/'
sequence_length = 30

label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in np.array([folder for folder in os.listdir(os.path.join(DATA_PATH, action)) if '.avi' not in folder]).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
X = np.array(sequences)
y = to_categorical(labels).astype(int)

# Split train and test set (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Split test set to test and validation set (10:10)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# 5 LSTM Model Traning

In [ ]:
log_dir = os.path.join('Logs/LSTM/')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
model.add(LSTM(32, return_sequences=True, activation='relu', input_shape=(sequence_length,1662)))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
# model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# model.fit(X_train, y_train, epochs=200, validation_data=(X_val, y_val), callbacks=[tb_callback])

Epoch 1/1000
21/21 [==============================] - 3s 48ms/step - loss: 3.1120 - categorical_accuracy: 0.0536 - val_loss: 3.0792 - val_categorical_accuracy: 0.0119
Epoch 2/1000
21/21 [==============================] - 1s 31ms/step - loss: 3.0415 - categorical_accuracy: 0.0565 - val_loss: 3.0457 - val_categorical_accuracy: 0.0119
Epoch 3/1000
21/21 [==============================] - 1s 31ms/step - loss: 2.9964 - categorical_accuracy: 0.0595 - val_loss: 2.9922 - val_categorical_accuracy: 0.0595
Epoch 4/1000
21/21 [==============================] - 1s 31ms/step - loss: 3.0782 - categorical_accuracy: 0.0655 - val_loss: 3.0855 - val_categorical_accuracy: 0.0119
Epoch 5/1000
21/21 [==============================] - 1s 27ms/step - loss: 2.9816 - categorical_accuracy: 0.0744 - val_loss: 2.9587 - val_categorical_accuracy: 0.0952
Epoch 6/1000
21/21 [==============================] - 1s 24ms/step - loss: 2.8594 - categorical_accuracy: 0.0789 - val_loss: 2.7473 - val_categorical_accuracy: 0.142

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 30, 32)            216960    
                                                                 
 lstm_3 (LSTM)               (None, 64)                24832     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 21)                693       
                                                                 
Total params: 248,725
Trainable params: 248,725
Non-trainable params: 0
_________________________________________________________________


# 6 evaluation

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)
accuracy_score(ytrue, yhat)

0.7261904761904762

In [ ]:
# model.save('LSTM200.h5')
# model.load_weights('LSTMopti2000.h5')

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 30, 32)            216960    
                                                                 
 lstm_3 (LSTM)               (None, 64)                24832     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 21)                693       
                                                                 
Total params: 248,725
Trainable params: 248,725
Non-trainable params: 0
_________________________________________________________________


# test

In [ ]:
# import cv2
# import numpy as np
# from PIL import ImageFont, ImageDraw, Image
# #######################################################################################################
# ## input here
# #######################################################################################################
# path = "E:/Downloads/ActionDetectionforSignLanguage-main/dataset/ทารก/1.avi"
# path = 0
# #######################################################################################################
# vdo = cv2.VideoCapture(path, cv2.CAP_DSHOW)
# sequence = []

# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     if vdo.isOpened():
#         chk_img, img = vdo.read()
#         while cv2.waitKey(10)!=27: #esc=27, space=32, backspace=8
#             chk_img, img = vdo.read()
#             if chk_img:
#                 image, results = mediapipe_detection(img, holistic)
#                 draw_styled_landmarks(image, results)
#                 keypoints = extract_keypoints(results)
#                 sequence.append(keypoints)
#                 sequence = sequence[-sequence_length:]
#                 if len(sequence) == sequence_length:
#                     res = model.predict(np.expand_dims(sequence, axis=0))[0]
#                     # print(actions[np.argmax(res)])
#                     text = str(actions[np.argmax(res)])
#                     font = ImageFont.truetype("./angsau.ttf", int(image.shape[0]*0.16))
#                     img_pil = Image.fromarray(image)
#                     draw = ImageDraw.Draw(img_pil)
#                     draw.text((int(image.shape[1]*0.15),int(image.shape[0]*0.02)),  text, font = font,)
#                     image = np.array(img_pil)
#                 cv2.imshow('OpenCV',image)
#     else:
#         print('video not found: please check video path')
#     cv2.destroyAllWindows()
#     if vdo.isOpened():
#         vdo.release()